# 导入模块

In [1]:
import numpy as np
import pandas as pd
import xlwings as xw
from openpyxl import load_workbook
import matplotlib.pyplot as plt
plt.rcParams["font.sans-serif"]=["SimHei"]
plt.rcParams["axes.unicode_minus"]=False

import warnings
warnings.filterwarnings("ignore")

import pymysql
from sqlalchemy import create_engine
import json
import datetime
from datetime import timedelta ,time, datetime
from dateutil.relativedelta import relativedelta
import re
import os
import glob
import sys
# 获取当前工作目录
current_dir = os.getcwd()
# 将模块所在的目录添加到 sys.path
module_dir = os.path.join(current_dir, '../Class_Model')
sys.path.append(module_dir)
from All_Class import All_Model, Week_Model, Data_Clean

In [2]:
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
# 实例化All_Model类
all_models = All_Model()
week_models = Week_Model()
clean = Data_Clean()

# 链接数据库

In [3]:
def query(sql,
          host="rr-wz9wx0w3yti9d4f6wro.mysql.rds.aliyuncs.com",
          user="ylc",
          password="1O8t5lcJ5aMhwwPEUUjS",
          database = '' ,
          port=3306
         ):   
    conn = pymysql.connect(
        host=host,
        user=user,
        port=port,
        password=password,
        max_allowed_packet=1073741824,
        charset="utf8")
    try:
        df = pd.read_sql(sql, con=conn)
        conn.close()
    except:
        print('error')
        conn.close()
        raise
    return df

# 导入数据

In [4]:
sql1 = ''' -- 订单&风控信息  近10日数据   
SELECT date(om.create_time) as create_date,om.create_time,om.id as order_id ,om.order_number,om.all_money 
,om.status, om.user_id
,case om.`status` when  1 then "待支付" when  2 then "待发货" when  3 then "待收货" when  4 then "租赁中" when  5 then "待归还" 
when  6 then "待商家收货" when  7 then "退押中" when  8 then "已完成" when  10 then "已退款" when  11 then "待退押金" when  12 then "待审核" 
when  13 then "订单取消" when  15 then "检测中" when  9999 then "逾期订单" end as status2 
,case when locate('租物',pa.name)>0 or locate('租物',cc.name)>0 or locate('芝麻',pa.name)>0 or locate('芝麻',cc.name)>0  then '芝麻租物' when locate('抖音',pa.name)>0 then '抖音渠道' when locate('搜索',cc.name)>0 then '搜索渠道' else '其他渠道' end as channel_type 
,tod.sku_attributes,tod.product_name,tod.new_actual_money
,case when  locate('租完即送',tod.sku_attributes)>0 then '租完即送' else '租物归还' end as back_type
,om.user_mobile,tmu.true_name,tmu.id_card_num
,top.total_describes,tor.decision_result,om.cancel_reason
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.traceid') end,'"','') as trace_id 
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.rejected') end,'"','') as rejected 
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.result') end,'"','') as result 
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.tips') end,'"','') as tips  
,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.is_vip') end,'"','') as is_vip
,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.status') end,'"','') as status_result
,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.queue_verify_type') end,'"','') as qvt_risk
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.queue_verify_type') end,'"','') as qvt_result

,cc.name as channel_name         -- 来源渠道
,cc.channel_type_id              -- 渠道id
,pa.name as activity_name        -- 活动名称
,om.merchant_id,om.merchant_name
,topay.total_freeze_fund_amount , tod.dy_order_item_json, pa.type
,om.buy_service_product,tso.status as service_status 
,om.order_method, om.activity_id, om.appid, tprm.max_overdue_days
, tor.update_time, tomt.reason, tprm.purchase_amount
from  db_digua_business.t_order  om
left join db_digua_business.t_postlease_receivables_monitoring tprm on tprm.order_id=om.id
left join db_digua_business.t_order_risk tor on om.id = tor.order_id
-- 备注信息合并 
left join ( SELECT  t.order_id,JSON_ARRAYAGG(t.describes) as total_describes from db_digua_business.t_order_personnel t   GROUP BY 1 ) top 
on om.id = top.order_id 
-- 服务信息
left join  db_digua_business.t_service_order tso  on om.id = tso.order_id 
-- 渠道名称
left join db_digua_business.t_channel cc on om.channel = cc.scene 
-- 活动名称
left join db_digua_business.t_platform_activity pa on om.activity_id = pa.id
-- 用户信息 
left join db_digua_business.t_member_user tmu on om.user_id = tmu.id
-- 商品信息
left join db_digua_business.t_order_details tod on om.id = tod.order_id
-- 免押信息  
left join (SELECT t.*,row_number() over(partition by t.order_id order by t.pay_date desc) as rn 
from db_digua_business.t_order_pay t 

where t.pay_type = 'ZFBYSQ' and t.item_type=1 and t.`status` in (2,5) and t.trade_no is not null )  topay 
on topay.order_id=om.id   and  topay.rn = 1 
-- 商家订单转移表
left join db_digua_business.t_order_merchant_transfer tomt on tomt.order_id=om.id


where om.user_mobile is not null 
and tmu.true_name not in ("刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华"
,"邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文",'张娜','罗文龙','孔靖','彭康力','何薪华','夏玥','潘佳','包闻天','方全龙','李楠','向圆圆','黄兰娟','林婉婷','廖丽敏','李巧玲','李巧凤','刘三妹','蔡斯静','陈宜诗','陈宝易','林寅钗','谢金凤','刘宏生','骆昌鑫','何静', '李珍珍')
-- and pa.type!=4
-- and  om.create_time >= DATE_ADD(CURRENT_DATE,INTERVAL -15 day )               -- 近15天数据
-- and  om.create_time <= DATE_ADD(CURRENT_DATE,INTERVAL -13 day )
and  DATE_FORMAT(om.create_time, '%Y-%m-%d')>='2025-01-01'
-- and  DATE_FORMAT(om.create_time, '%Y-%m-%d')<='2025-02-28'
'''

df_order = query(sql1)
df_order.shape

(328736, 44)

In [5]:
sql_risk = ''' -- risk等级
select trace_id, id_card as id_card_r, time
, replace(case when JSON_VALID(data) THEN JSON_EXTRACT(data, '$.status') end,'"','') as status_r 
, replace(case when JSON_VALID(data) THEN JSON_EXTRACT(data, '$.is_model') end,'"','') as is_model
from db_credit.risk
'''
df_risk = query(sql_risk)

In [6]:
sql3 = '''
SELECT risk_trace_id trace_id, id_card, time, replace(case when JSON_VALID(data) THEN JSON_EXTRACT(data, '$.status') end,'"','') as status   FROM `db_credit`.risk_examine
'''
df_risk_examine = query(sql3)
df_re = df_risk_examine.copy()
df_re.loc[:, 'time_re'] = pd.to_datetime(df_re.time.dt.date,errors="coerce")
df_re.rename(columns={'id_card': 'id_card_re', 'status': 'status_re'}, inplace=True)

In [7]:
sql_ra = ''' -- 996强拒表
select order_id, time, replace(case when JSON_VALID(result) THEN JSON_EXTRACT(result, '$.status') end,'"','') as status_ra  from db_credit.risk_alipay_interactive_prod_result
'''
df_ra = query(sql_ra)
df_ra.loc[:, 'time_ra'] = pd.to_datetime(df_ra.time.dt.date,errors="coerce")
df_ra = df_ra.sort_values(by='time', ascending=False).groupby('order_id').head(1)

# 合并数据

In [8]:
df = df_order.copy()
df = df[df.type!=4]
df.order_id.notnull().sum()

np.int64(328430)

# 处理日期

In [9]:
df["下单日期"]=df["create_time"].dt.date
df["下单日期"]=pd.to_datetime(df["下单日期"],errors="coerce")
df["月份"]=df["下单日期"].dt.month
df["下单月份"]=df["create_time"].dt.strftime('%Y-%m')
df['年份'] = df["下单日期"].dt.year
df['hour'] = df['create_time'].dt.hour
df['拒绝理由'] = df["rejected"].str.replace("[","").str.replace("]","").str.replace('"','')
t_date = datetime.today().strftime('%Y%m%d')

# 渠道补充

In [10]:
qudao_name_df = pd.read_excel('F:/myfile/租后数据/历史订单渠道名称补充.xlsx')
qudao_name_df.drop(columns=['渠道id','下单时间'],inplace = True)
qudao_name_df.rename(columns={'订单号': '订单号1'}, inplace=True)
df =  df.merge(qudao_name_df,left_on = 'order_number',right_on = '订单号1',how = 'left')
df["channel_name"] = np.where(df.渠道名称.notnull(),df.渠道名称,df["channel_name"])

# 处理备注信息

In [11]:
df = df[df['sku_attributes'].notnull()]
df["取消原因"]=df["total_describes"].str.split("客户申请取消：").str[1].str.split("$").str[0].str.strip()
df["电审拒绝原因"]=df["total_describes"].str.split("审核不通过：").str[1].str.split("$").str[0].str.strip()
df["无法联系原因"]=df["total_describes"].str.split("用户无法联系：").str[1].str.split("$").str[0].str.strip()
df.loc[:,"商品类型"]=np.where(df["product_name"].str.contains('99新')|df["product_name"].str.contains('95新')|df["product_name"].str.contains('准新'),"二手","全新")  ##  准新
df.loc[:,"租赁方案"]=np.where(df["sku_attributes"].str.contains('租完即送'),"租完即送","租完归还")
df.loc[:,"押金类型"]=np.where(df["total_freeze_fund_amount"]>0,"部分免押","全免押")
df.loc[:,"优惠券使用否"]=np.where((df["new_actual_money"]-df["all_money"]>0)&(df["租赁方案"]=='租完即送'),"已使用","未使用")
df.loc[:,"是否进行预授权"]=np.where(df["total_freeze_fund_amount"].isnull(),"未预授权","已预授权")

def getcolor(s):
    color_list=json.loads(s)
    for j in range(0,len(color_list)):
        if color_list[j]["key"]=="颜色":
            return color_list[j]["value"]
df.loc[:,"颜色"]=df.apply(lambda x:getcolor(x["sku_attributes"]),axis=1)
df["颜色"].value_counts()  


def getneicun(s):
    color_list=json.loads(s)
    for j in range(0,len(color_list)):
        if color_list[j]["key"]=="内存":
            return color_list[j]["value"]

df.loc[:,"内存"]=df.apply(lambda x: getneicun(x["sku_attributes"]),axis=1)
df["内存"].value_counts() 

df.loc[:,"进件"]=np.where((df["status2"]=="待支付")|(df["status2"]=="订单取消"),"未进件","进件")
df["进件"].value_counts()


df.loc[:,"来源渠道"]=df["channel_name"].fillna("未知渠道")

df.loc[:,"归属渠道"]=df.apply(lambda x: clean.qudao_type(x["来源渠道"],x["activity_name"],x["order_method"], x['channel_type_id']),axis=1)

# 订单去重

In [12]:
dict_status_code = {
    "订单取消": 1,
    "待支付": 2,
    "已退款": 3, 
    "待审核": 4,
    "待发货": 5,
    "待收货": 6,
    "租赁中": 7,
    "已完成": 8
}
df["状态编码"]=df["status2"].map(dict_status_code)


df.sort_values(by=["下单日期","状态编码"],inplace=True)


#删除重复单号
df.drop_duplicates(subset=["order_id"],inplace=True)
#删除身份证空值行
df.dropna(subset=["id_card_num"],axis=0,inplace=True)
#去刷单订单
df.drop(df[df['total_describes'].str.contains(pat='panli',regex=False)==True].index,inplace=True)
df.drop(df[df['activity_name']=="1000单秘密计划"].index,inplace=True)
df.drop(df[df['activity_name']=="1000单秘密计划-无优惠"].index,inplace=True)
df.drop(df[df['activity_name']=="1000单曙光计划"].index,inplace=True)
df.drop(df[df['activity_name']=="线下门店3个月试行"].index,inplace=True)
#删除身份证空值行
df.dropna(subset=["status2"],axis=0,inplace=True)


#删除重复订单
df.drop_duplicates(subset=["order_id"],inplace=True)
df.drop_duplicates(subset=["true_name","user_mobile","id_card_num","下单日期"],keep="last",inplace=True)


df.drop(df[df['true_name'].isin(["刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华"
,"邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文"])].index,inplace=True)

# 定义状态

In [13]:
# note 2025-02-24添加
df = df.merge(df_risk[['trace_id', 'status_r']], on='trace_id', how='left').merge(df_re[['trace_id', 'status_re']], on='trace_id', how='left').merge(df_ra[['order_id', 'time_ra', 'status_ra']], left_on=['order_id', '下单日期'], right_on=['order_id', 'time_ra'], how='left')

# 判断 前置拦截   机审强拒   出库前风控强拒
df['是否前置拦截'] = np.where((df.result.str.contains('id_card不得为空') )|(df.result.str.contains('mobile校验不通过')) |(df.result.str.contains('name校验不通过'))
                        |(df.result.str.contains('年龄超过49岁或低于18岁'))|(df.result=='风控未通过')|(df.result=='黑名单用户'),1,0)
# note 2025-02-24修改
df['是否机审强拒'] = np.where(df.status_r=='1', 1, 0)
df['是否出库前风控强拒'] = np.where((df.status_re=='1')|(df.status_ra=='1'), 1, 0)
df.loc[:,"审核状态"]=df.apply(lambda x: clean.reject_type(x["拒绝理由"],x["进件"],x["电审拒绝原因"],x["取消原因"],x["status2"],x["无法联系原因"],x["total_describes"],x['是否前置拦截'],x['是否机审强拒'],x['是否出库前风控强拒']),axis=1) 


# 保留商家

In [14]:
df_contain = df.copy()
# 调用Data_Clean类中的drop_merchant函数
df = clean.drop_merchant(df)

# 设置节点状态

In [15]:
df["待审核"] = np.where(df["审核状态"]=='待审核',1,0)
df["前置拦截"] = np.where(df["审核状态"]=='前置拦截',1,0)
df["人审拒绝"] = np.where(df["审核状态"]=='人审拒绝',1,0)
df["客户取消"] = np.where(df["审核状态"]=='客户取消',1,0)

df["无法联系"] = np.where(df["审核状态"]=='无法联系',1,0)
df["是否进件"] = np.where(df["进件"]=='进件',1,0)
df["是否出库"] = np.where(df["status"].isin([2,3,4,5,6,8,15]),1,0)

df["进件前取消"] = np.where(df["审核状态"]=='进件前取消',1,0)


df['是否出库'] = np.where((df['人审拒绝']==0)&(df['客户取消']==0)&(df['无法联系']==0)&(df['待审核']==0)&(df['是否出库']==1), 1, 0)
df["出库前风控强拒"] = np.where((df["审核状态"]=='出库前风控强拒')&(df['是否出库']==0),1,0)
df["机审强拒"] = np.where((df["审核状态"]=='机审强拒')&(df['是否出库']==0),1,0)
df['机审通过件'] = np.where((df['是否进件']==1)&(df['机审强拒']==0), 1, 0)
df['风控通过件'] = np.where((df['是否进件']==1)&(df['机审强拒']==0)&(df['人审拒绝']==0), 1, 0)

df['已退款'] = np.where((df['风控通过件']==1)&(df['审核状态']=='已退款'), 1, 0)

df['是否二手'] = np.where(df['product_name'].str.contains(r'99新|95新|准新|90新'), 1, 0)

# 导入台账数据

In [16]:
f_path_ck = "F:/myfile/p站数据/台账数据/维客壹佰2025年台账.xlsx"
df_ck = pd.read_excel(f_path_ck,sheet_name="2025") 
dfck = pd.merge(df_ck,df,left_on="订单号", right_on="order_number")
dfck.drop_duplicates(subset=["订单号"],inplace=True)
dfck.drop(dfck[dfck["status2"]=="已退款"].index,inplace=True)

# 删除 露营设备 出库 
dfck.drop(dfck[dfck["类型"]=="露营设备"].index,inplace=True)

# 将序列号转换为日期
base_date = pd.Timestamp('1899-12-30')  # Excel的基准日期
dfck['日期'] = base_date + pd.to_timedelta(dfck['日期'], unit='D')

# 剔除拒量

In [17]:
df2 = df.copy()
df2 = clean.drop_rejected_merchant(df2)

# 自由模型 2311

In [18]:
sql_2311='''-- 风控-自有模型-ascore2311 
SELECT trace_id, score
from db_credit.risk_private_model
;
'''
mx_2311_df =query(sql_2311)
mx_2311_df['model_score_2311'] = pd.to_numeric(mx_2311_df['score'], errors='coerce')

# 自有模型 2401

In [19]:
sql_2401='''-- 风控-自有模型-2401
SELECT trace_id,  score
from db_credit.risk_private_model_2401
;
'''
mx_2401_df =query(sql_2401)
mx_2401_df['model_score_2401'] = pd.to_numeric(mx_2401_df['score'], errors='coerce')

# 自有模型-银联

In [20]:
sql_yl='''-- 风控-自有模型-yl
SELECT trace_id,  score
from db_credit.risk_private_model_yl
;
'''
mx_yl_df =query(sql_yl)
mx_yl_df['model_score_yl'] = pd.to_numeric(mx_yl_df['score'], errors='coerce')

# 风控-自有模型-2501

In [21]:
sql_base = ''' -- 风控-自有模型-2501
select trace_id, score as score_base from db_credit.risk_private_model_2501_base
'''
df_base = query(sql_base)

# 关联模型

In [22]:
# 获取拒量
df2_jl = df[df.tips.str.contains(r'策略2412|命中自有模型回捞策略')==True]
df2_jl.loc[:, '机审强拒'] = np.where(df2_jl.qvt_risk=='1', 1, 0)
df2_jl.loc[:, '出库前风控强拒'] = np.where(((df2_jl.qvt_risk == '0')|(df2_jl.qvt_risk.isna()))&(df2_jl.qvt_result == '1'), 1, 0)
df2_jl_group = df2_jl.groupby('下单日期').agg({'出库前风控强拒': 'sum'})

In [23]:
# 获取免审进件非抖音的数据，并关联旧模型
df2_new = df2[(df2.是否进件==1)&(df2.归属渠道!='抖音渠道')]
df_merge = df_risk[['trace_id', 'is_model']].merge(mx_2311_df, on='trace_id', how='left').merge(mx_2401_df, on='trace_id', how='left').merge(mx_yl_df, on='trace_id', how='left')

In [24]:
# 限制时间并关联新模型
df2_model_score = df2_new[(df2_new.下单日期>='2025-02-19')].merge(df_merge, on='trace_id', how='left').merge(df_base, on='trace_id', how='left')
df2_model_score['score_base'] = df2_model_score.score_base.astype(float)
# df2_model_score[['下单日期', 'order_number', 'model_score_2311', 'model_score_2401', 'model_score_yl', 'score_base', 'is_model', '拒绝理由', '归属渠道']]

## 芝麻租物
1.cutoff
租物渠道：评分>=0且评分<=14.104，免人审；评分>14.104且评分<=18.1312，信
审；评分>18.1312，强拒



In [25]:
def cutoff(df, model, score1, score2, channel_name1,  channel_name2=None):
    if channel_name2==None:
        df_zw = df[(df.归属渠道==channel_name1)&(df.is_model==model)]
    else:
        df_zw = df[(df.归属渠道!=channel_name1)&(df.归属渠道!=channel_name2)&(df.is_model==model)]
    if model=='1':
        df_zw.loc[:, '状态'] = np.where((df_zw.model_score_2311>=0)&(df_zw.model_score_2311<=score1), '免人审',
                                            np.where((df_zw.model_score_2311>score1)&(df_zw.model_score_2311<=score2), '信审', '强拒'))
    elif model=='2':
        df_zw.loc[:, '状态'] = np.where((df_zw.model_score_yl>=0)&(df_zw.model_score_yl<=score1), '免人审',
                                            np.where((df_zw.model_score_yl>score1)&(df_zw.model_score_yl<=score2), '信审', '强拒'))
    elif model=='3':
        df_zw.loc[:, '状态'] = np.where((df_zw.model_score_2401>=0)&(df_zw.model_score_2401<=score1), '免人审',
                                            np.where((df_zw.model_score_2401>score1)&(df_zw.model_score_2401<=score2), '信审', '强拒'))
    elif model=='4':
        df_zw.loc[:, '状态'] = np.where((df_zw.score_base>=0)&(df_zw.score_base<=score1), '免人审',
                                            np.where((df_zw.score_base>score1)&(df_zw.score_base<=score2), '信审', '强拒'))
    df_zw_group = df_zw.groupby(['is_model', '状态']).agg({'是否进件': 'sum', '机审强拒': 'sum', '机审通过件':'sum', '风控通过件': 'sum', '出库前风控强拒': 'sum', '是否出库': 'sum'}).rename(columns={'是否进件': '进件数', '是否出库': '出库'})

    return df_zw, df_zw_group

In [26]:
# 模型4 2501
df2_model_score_zw4, df2_model_score_zw4_group = cutoff(df2_model_score, '4', 14.104, 18.1312, '芝麻租物')
# df2_model_score_zw4_group

# 模型1 2311
df2_model_score_zw1, df2_model_score_zw1_group = cutoff(df2_model_score, '1', 18.321, 23.297, '芝麻租物')
# df2_model_score_zw1_group

# 模型2 yl
df2_model_score_zw2, df2_model_score_zw2_group = cutoff(df2_model_score, '2', 17.55, 23.382, '芝麻租物')
# df2_model_score_zw2_group

# 模型3 2401
df2_model_score_zw3, df2_model_score_zw3_group = cutoff(df2_model_score, '3', 11.04, 16.0, '芝麻租物')
# df2_model_score_zw3_group

df_concat_zw = pd.concat([df2_model_score_zw1_group,df2_model_score_zw2_group,df2_model_score_zw3_group,df2_model_score_zw4_group])

## 非芝麻租物 非搜索渠道
非租物渠道：评分>=0且评分<=13.79，免人审；评分>13.79且评分<=18.098，信
审；评分>18.098，强拒

In [27]:
# 模型4 2501
df2_model_score_fzw4, df2_model_score_fzw4_group = cutoff(df2_model_score, '4', 13.79, 18.098, '芝麻租物', '搜索渠道')
# df2_model_score_fzw4_group

# 模型1 2311
df2_model_score_fzw1, df2_model_score_fzw1_group = cutoff(df2_model_score, '1', 18.321, 23.297, '芝麻租物', '搜索渠道')
# df2_model_score_fzw1_group

# 模型2 yl
df2_model_score_fzw2, df2_model_score_fzw2_group = cutoff(df2_model_score, '2', 17.55, 23.382, '芝麻租物', '搜索渠道')
# df2_model_score_fzw2_group

# 模型3 2401
df2_model_score_fzw3, df2_model_score_fzw3_group = cutoff(df2_model_score, '3', 11.04, 16.0, '芝麻租物', '搜索渠道')
# df2_model_score_fzw3_group

df_concat_fzw = pd.concat([df2_model_score_zw1_group,df2_model_score_zw2_group,df2_model_score_zw3_group,df2_model_score_zw4_group])

# with pd.ExcelWriter(f'F:/需求/总需求/cutoff分值统计_{t_date}.xlsx', engine='xlsxwriter') as writer:
#     df_concat_zw.to_excel(writer, sheet_name='芝麻租物')
#     df_concat_fzw.to_excel(writer, sheet_name='非芝麻搜索')

# 旧模型

In [109]:
df2_model_score_old = df2_new[(df2_new.下单日期>='2025-01-01')&(df2_new.下单日期<='2025-01-31')].merge(df_merge, on='trace_id', how='left')
df2_model_score_old[['下单日期', 'order_number', 'model_score_2311', 'model_score_2401', 'model_score_yl', 'is_model', '拒绝理由', '归属渠道']]

,下单日期,order_number,model_score_2311,model_score_2401,model_score_yl,is_model,拒绝理由,归属渠道
0,2025-01-01,A202501010015178,NaN,NaN,NaN,None,命中非免押用户拒绝,芝麻租物
1,2025-01-01,A2025010100333630,18.707926,9.705912,28.676603,2,命中自有模型强拒,芝麻租物
2,2025-01-01,A2025010100565540,22.449318,17.561240,25.411859,1,,单人聊天会话中的小程序消息卡片（分享）
3,2025-01-01,A2025010100573841,20.284838,9.276167,22.047317,3,,芝麻租物
4,2025-01-01,A2025010101391118,NaN,NaN,NaN,None,命中非免押用户拒绝,搜索渠道
...,...,...,...,...,...,...,...,...
12563,2025-01-31,A2025013122443313,15.317005,10.637538,17.557327,2,"命中融安分低于680强拒(芝麻综合风险等级0豁免), 评分等级低于D,命中拒绝",芝麻租物
12564,2025-01-31,A2025013123532439,19.694709,11.496813,30.872897,3,"命中蚁盾分>50, 命中被执行人或曝光台, 命中青云分<500",搜索渠道
12565,2025-01-31,A202501310146535,13.628374,5.764763,13.620016,2,,搜索渠道
12566,2025-01-31,A202501310303332,21.168919,9.586799,18.194756,2,,芝麻租物


# 新模型通过率

In [118]:
df2_model_score_4 = df2_model_score[df2_model_score.is_model=='4']
df2_model_score_group = df2_model_score_4.groupby('下单日期').agg({'机审通过件': 'sum', '人审拒绝': 'sum', '风控通过件': 'sum', '客户取消': 'sum', '无法联系': 'sum', '待审核': 'sum', '出库前风控强拒': 'sum', '是否出库': 'sum', '已退款': 'sum'}).rename(columns={'是否进件': '进件数', '是否出库': '出库'})
df2_model_score_group['客户取消'] = df2_model_score_group.客户取消+df2_model_score_group.已退款
df2_model_score_group['风控通过件'] = df2_model_score_group.风控通过件+df2_jl_group.出库前风控强拒
df2_model_score_group['机审通过件'] = df2_model_score_group.机审通过件+df2_jl_group.出库前风控强拒
df2_model_score_group['出库前风控强拒'] = df2_model_score_group.出库前风控强拒+df2_jl_group.出库前风控强拒
df2_model_score_group.loc[:, '进件出库率'] = (df2_model_score_group.出库/df2_model_score_group.机审通过件).map(lambda x: format(x, '.2%'))
df2_model_score_group
# df2_model_score_4[df2_model_score_4.风控通过件==1].to_excel('F:/xxx.xlsx')

,机审通过件,人审拒绝,风控通过件,客户取消,无法联系,待审核,出库前风控强拒,出库,已退款,进件出库率
下单日期,,,,,,,,,,
2025-02-19,70.0,17,53.0,19,0,0,12.0,22,4,31.43%
2025-02-20,85.0,16,69.0,24,0,0,18.0,27,6,31.76%
2025-02-21,80.0,6,74.0,33,1,0,15.0,25,10,31.25%
2025-02-22,48.0,6,42.0,12,0,0,10.0,20,4,41.67%
2025-02-23,68.0,7,61.0,25,0,0,8.0,28,6,41.18%
2025-02-24,92.0,10,82.0,29,5,0,20.0,28,9,30.43%
2025-02-25,101.0,8,93.0,38,2,0,18.0,35,10,34.65%
2025-02-26,101.0,15,86.0,28,3,0,20.0,35,10,34.65%
2025-02-27,81.0,9,72.0,29,2,0,11.0,30,9,37.04%
